In [ ]:
import anndata as ad
import pandas as pd
import numpy as np
import os

In [142]:
#ignore extra tfs from decoupler while writing script 

def tf_activity_analysis (anndataobject, tf_activities = None, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)

    #skipped tf activities part. ignore extra tf data from decoupler

    anndataobject.obs["condition"] = arguments_list["condition"] 
    anndataobject.obs["cell_type"] = arguments_list["celltype"]
    anndataobject.obs["comparison_list"] = arguments_list["comparison_list"]

    if not np.isnan(arguments_list["comparison_list"]):
        if len(arguments_list["comparison_list"]) > 0 & len(anndataobject.obs["comparison_list"]) < 2:
            arguments_list["comparison_list"] <- np.nan
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if np.isnan(arguments_list["comparison_list"]):
        anndataobject.uns["tf_annotation"] = pd.DataFrame({"result_list" : [],
        "gene_expression_list" : [],
        "CTR_cluster_list" : [],
        "intranet_cluster_list" : []})

    #anndataobject_list = split by condition, skipped for now
    sub_object = anndataobject

    #NOT THE SAME RESULTS AS R AVERAGE EXPRESSION
    sub_object.layers["Average_Expression"] = AverageExpression(sub_object, celltype = "new_annotation")

    #tf_activity_scores = get_significant_tfs_single(sub_object, name, tf_path, pval = arguments_list["pval", log2fc = arguments_list["log2fc"]])
    

    return(sub_object)

In [139]:
anndataobject.obs["cell_type"] = arguments_list["celltype"]
anndataobject

AnnData object with n_obs × n_vars = 498 × 11426
    obs: 'nCount_RNA', 'nFeature_RNA', 'Species', 'protocol', 'name', 'percent.mt', 'percent.rb', 'S.Score', 'G2M.Score', 'Phase', 'CC.Difference', 'integrated_snn_res.0.5', 'seurat_clusters', 'integrated_snn_res.0.4', 'annotation', 'new_annotation', 'cell_type'
    var: 'name'
    obsm: 'X_pca', 'X_umap'

In [145]:
anndataobject.obs

,nCount_RNA,nFeature_RNA,Species,protocol,name,percent.mt,percent.rb,S.Score,G2M.Score,Phase,CC.Difference,integrated_snn_res.0.5,seurat_clusters,integrated_snn_res.0.4,annotation,new_annotation,cell_type
H18_19086_TB_AAATGCCAGTGTACGG-1,271.0,192,Human Biopsy,control,H18_19086_TB,2.214022,32.841328,-0.013804,-0.010867,G1,-0.002936,6,3,3,Megakaryocyte,Megakaryocyte,None
H18_19086_TB_AACTCTTCATCGTCGG-1,237.0,180,Human Biopsy,control,H18_19086_TB,3.797468,33.333333,-0.030984,-0.074486,G1,0.043502,5,2,2,Fibroblast,Fibroblast,None
H18_19086_TB_AATCGGTAGAGAGCTC-1,403.0,221,Human Biopsy,control,H18_19086_TB,9.925558,6.203474,-0.030863,-0.066165,G1,0.035302,2,1,1,MSC,MSC,None
H18_19086_TB_ACACCAAGTAAGTGTA-1,224.0,175,Human Biopsy,control,H18_19086_TB,0.446429,28.571429,-0.024614,0.002616,G2M,-0.027231,6,3,3,Megakaryocyte,Megakaryocyte,None
H18_19086_TB_ACAGCCGTCATCTGTT-1,218.0,161,Human Biopsy,control,H18_19086_TB,0.917431,29.357798,-0.023215,0.096029,G2M,-0.119244,6,3,3,Megakaryocyte,Megakaryocyte,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
H18_19892_TB_CAGCAGCGTAACGCGA-1,52.0,45,Human Aspirate,"PMF,MF2",H18_19892_TB,1.562500,10.937500,-0.003332,-0.025919,G1,0.022587,12,0,0,Neural,Neural,None
H18_19892_TB_GGTGCGTGTTTGTTTC-1,57.0,54,Human Aspirate,"PMF,MF2",H18_19892_TB,2.631579,13.157895,-0.006548,-0.022244,G1,0.015696,12,0,0,Neural,Neural,None
H18_19892_TB_TACCTATCACAGGCCT-1,192.0,129,Human Aspirate,"PMF,MF2",H18_19892_TB,0.127443,1.571793,-0.005028,0.093877,G2M,-0.098905,12,0,0,Neural,Neural,None
H18_19892_TB_TAGAGCTGTTAGATGA-1,54.0,46,Human Aspirate,"PMF,MF2",H18_19892_TB,0.000000,36.363636,-0.006616,0.171603,G2M,-0.178219,12,0,0,Neural,Neural,None


In [144]:
sub_object_avg = tf_activity_analysis(anndataobject= "/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/anndata_object.h5ad", arguments_list= {"out_path" : "folder", "celltype" : None, "condition" : None, "organism" : None, "comparison_list" : None, "logfc" : None, "pval" : None, "reg" : "/home/larissa/Documents/LR2TF_HiWi/human_dorothea_reg.csv"})

Please provide the name of the metadata field containing cell type annotations
Please provide the name of the metadata field containing condition annotations


/tmp/ipykernel_434287/2594499671.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_obs = obs.groupby(level=0).mean()


In [135]:
arguments = {"out_path" : "folder", "celltype" : None, "condition" : None, "organism" : None, "comparison_list" : None, "logfc" : None, "pval" : None, "reg" : "/home/larissa/Documents/LR2TF_HiWi/human_dorothea_reg.csv"}
arguments_list = validate_input_arguments(arguments)
arguments_list["comparison_list"] 

Please provide the name of the metadata field containing cell type annotations
Please provide the name of the metadata field containing condition annotations


nan

In [ ]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight"in arguments_list["reg"]:
        raise Exception("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns source, target and weight!")
    
 
   
    return(arguments_list)



In [143]:
def AverageExpression(anndataobject, celltype = None):
    gene_ids = anndataobject.var.index.values
    #cluster line necessary if not even returned?
    clusters = anndataobject.obs[celltype].cat.categories
    obs = anndataobject[:,gene_ids].X.toarray()
    obs = pd.DataFrame(obs,columns=gene_ids,index= anndataobject.obs[celltype])
    average_obs = obs.groupby(level=0).mean()
    average_obs.T.to_csv("average.csv")

    return obs

#https://github.com/scverse/scanpy/issues/336
#https://github.com/scverse/scanpy/issues/181

In [159]:
gene_ids = anndataobject.var.index.values
clusters = anndataobject.obs["new_annotation"].cat.categories
obs = anndataobject[:,gene_ids].X.toarray()
obs = pd.DataFrame(obs,columns=gene_ids,index= anndataobject.obs["new_annotation"])
average_obs = obs.groupby(level=0).mean()
average_obs

,FO538757.2,RP11-206L10.9,SAMD11,NOC2L,HES4,ISG15,TNFRSF18,TNFRSF4,SDF4,B3GALT6,...,KIF21A,RP11-412D9.4,SNHG23,ASPA,SLFN12,SMYD5,CCDC85C,CPNE2,TMEM206,CCDC9
new_annotation,,,,,,,,,,,,,,,,,,,,,
Neural,0.045191,0.00000,0.0,0.000000,0.000000,1.216800,0.000000,0.000000,0.000000,0.000000,...,0.025288,0.0,0.0,0.094721,0.0,0.0,0.0,0.0,0.0,0.0
MSC,0.365838,0.02069,0.0,0.078729,0.252494,0.926320,0.000000,0.000000,0.182018,0.052287,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Fibroblast,0.043312,0.03493,0.0,0.123824,0.000000,0.254106,0.000000,0.000000,0.223904,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Megakaryocyte,0.308894,0.00000,0.0,0.051798,0.074781,0.748965,0.236492,0.158068,0.163902,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Myeloid,0.112166,0.00000,0.0,0.053173,0.000000,1.054567,0.000000,0.000000,0.074715,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight"in arguments_list["reg"]:
        raise Exception("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns source, target and weight!")
    
 
   
    return(arguments_list)

